In [2]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [3]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai.embeddings import OpenAIEmbeddings

### FAISS 벡터 저장소 생성 (from_texts)

`from_texts()`  텍스트 리스트와 임베딩 함수를 사용하여 FAISS 벡터 저장소를 생성합니다. 

**매개변수**

- texts (List[str]): 벡터 저장소에 추가할 텍스트 리스트
- embedding (Embeddings): 사용할 임베딩 함수
- metadatas (Optional[List[dict]]): 메타데이터 리스트. 기본값은 None
- ids (Optional[List[str]]): 문서 ID 리스트. 기본값은 None
- **kwargs: 추가 키워드 인자

In [4]:
# 리스트에 데이터로 문자열 텍스트를 넣어서 만든다.
db = FAISS.from_texts(
    ["제 이름은 이인환입니다.", "저는 컴퓨터프로그래밍을 교육하는 강사입니다."],
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    metadatas=[{"source": "텍스트문서"}, {"source": "텍스트문서"}],
    ids=["doc1", "doc2"],
)

In [5]:
db.docstore._dict

{'doc1': Document(metadata={'source': '텍스트문서'}, page_content='제 이름은 이인환입니다.'),
 'doc2': Document(metadata={'source': '텍스트문서'}, page_content='저는 컴퓨터프로그래밍을 교육하는 강사입니다.')}

### **텍스트로부터 추가 (add_texts)**

add_texts() 텍스트를 임베딩하고 벡터 저장소에 추가하는 기능을 제공합니다.
<br>

**매개변수**

- texts (Iterable[str])               : 벡터 저장소에 추가할 텍스트 이터러블
- metadatas (Optional[List[dict]])    : 텍스트와 연관된 메타데이터 리스트 (선택적)
- **kwargs                            : 추가 키워드 인자

**동작**
<br>

1. 입력받은 텍스트 이터러블을 리스트로 변환합니다.
2. _embed_documents 메서드를 사용하여 텍스트를 임베딩합니다.
3. __add 메서드를 호출하여 임베딩된 텍스트를 벡터 저장소에 추가합니다.


In [6]:
# 데이터 추가
db.add_texts(
    ["이번엔 텍스트 데이터를 추가합니다.", "추가한 2번째 텍스트 데이터 입니다."],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["new_doc2", "new_doc3"],
)

['new_doc2', 'new_doc3']

In [7]:
# 데이터 아이디
db.index_to_docstore_id

{0: 'doc1', 1: 'doc2', 2: 'new_doc2', 3: 'new_doc3'}

### **문서 삭제 (Delete Documents)**

`delete()` 벡터 저장소에서 지정된 ID에 해당하는 문서를 삭제하는 기능을 제공합니다.
<br>

**매개변수**

- ids (Optional[List[str]]): 삭제할 문서의 ID 리스트
- **kwargs: 추가 키워드 인자 (이 메서드에서는 사용되지 않음)

In [8]:
# 삭제할 데이터를 추가
ids = db.add_texts(
    ["삭제용 데이터를 추가합니다.", "2번째 삭제용 데이터입니다."],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["delete_doc1", "delete_doc2"],
)

In [10]:
print(ids)  # 삭제할 id 를 확인

['delete_doc1', 'delete_doc2']


In [11]:
db.delete(ids)  # id 로 삭제

True

In [12]:
# 삭제된 결과를 출력
db.index_to_docstore_id

{0: 'doc1', 1: 'doc2', 2: 'new_doc2', 3: 'new_doc3'}

### **로컬 저장 (Save Local)**

`save_local()` FAISS 인덱스, 문서 저장소, 그리고 인덱스-문서 ID 매핑을 로컬 디스크에 저장하는 기능을 제공합니다.
<br>

**매개변수**

- `folder_path` (str): 저장할 폴더 경로
- `index_name` (str): 저장할 인덱스 파일 이름 (기본값: "index")

**동작**

1. 지정된 폴더 경로를 생성합니다 (이미 존재하는 경우 무시).
2. FAISS 인덱스를 별도의 파일로 저장합니다.
3. 문서 저장소와 인덱스-문서 ID 매핑을 pickle 형식으로 저장합니다.

In [13]:
# 저장
db.save_local(folder_path="./faiss_db", index_name="faiss_index")

### **로컬에서 불러오기 (Load Local)**

`load_local()` 로컬 디스크에 저장된 FAISS 인덱스, 문서 저장소, 그리고 인덱스-문서 ID 매핑을 불러오는 기능을 제공합니다.
<br>

**매개변수**

- `folder_path` (str): 불러올 파일들이 저장된 폴더 경로
- `embeddings` (Embeddings): 쿼리 생성에 사용할 임베딩 객체
- `index_name` (str): 불러올 인덱스 파일 이름 (기본값: "index")
- `allow_dangerous_deserialization` (bool): pickle 파일 역직렬화 허용 여부 (기본값: False)

In [15]:
# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="./faiss_db",                                           # 불러올 파일들이 저장된 폴더 경로
    index_name="faiss_index",                                           # 불러올 인덱스 파일 이름
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),        # 임베딩 객체
    allow_dangerous_deserialization=True,                               # pickle 파일 역직렬화 허용
)

In [20]:
print(loaded_db.index_to_docstore_id)       # 로드된 데이터를 확인

{0: 'doc1', 1: 'doc2', 2: 'new_doc2', 3: 'new_doc3'}


In [21]:
retriever = loaded_db.as_retriever()        # 검색기로 변환

In [24]:
retriever.invoke('컴퓨터프로그래밍?')

[Document(metadata={'source': '텍스트문서'}, page_content='저는 컴퓨터프로그래밍을 교육하는 강사입니다.'),
 Document(metadata={'source': 'mydata.txt'}, page_content='이번엔 텍스트 데이터를 추가합니다.'),
 Document(metadata={'source': 'mydata.txt'}, page_content='추가한 2번째 텍스트 데이터 입니다.'),
 Document(metadata={'source': '텍스트문서'}, page_content='제 이름은 이인환입니다.')]